In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter

In [2]:
end = datetime.today()
start = end - relativedelta(years=2)

In [3]:
end.strftime('%Y-%m-%d')

'2022-06-01'

In [4]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [5]:
sp500 = SP500ticker_list['Symbol']
sp600 = SP600ticker_list['Ticker symbol']
sp400 = SP400ticker_list['Ticker symbol']


In [6]:
len(sp500)+len(sp600)+len(sp400)

1505

In [7]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

504
601
400


In [8]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

504
601
400


In [9]:
len(sp1500_index_df)-len(SP1500)

0

In [10]:
def getStock(npa):
    #position = npa[0]
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = yf.download(symbol, start=start_,end=end_)
    #data_['Sector'] = tickerdata.info['sector']
    #data_['Symbol'] = symbol
    #data_['Market Cap'] = data.get_quote_yahoo(names[n])['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [11]:
def getMarketCap(npa):
    symbol = npa[0]

    #start_=npa[2]
    #end_=npa[3]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = data.get_quote_yahoo(symbol)['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [12]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata.info['sector']])

In [13]:
def getFinObject(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata])

In [51]:
def calculateMinMax(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Close'][[-1]].index[0] - relativedelta(years=1)
    
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = np.max(pd.DataFrame(data_df)['High'])
    min_2y = np.min(pd.DataFrame(data_df)['Low'])
    quantiles_close_2y = pd.DataFrame(data_df)['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    std_2y = np.std(pd.DataFrame(data_df)['Close'])
    mean_2y = np.mean(pd.DataFrame(data_df)['Close'])
    
    max_1y = np.max(past_year['High'])
    min_1y = np.min(past_year['Low'])
    quantiles_close_1y = pd.DataFrame(data_df)['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    std_1y = np.std(past_year['Close'])
    mean_1y = np.mean(past_year['Close'])
    risk_trend = data_df['risk_per_share'][-1]
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend])

In [15]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [16]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getSector(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getSector, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [17]:
len(np.unique([r[0] for r in results_sectors]))

1148

In [18]:
sectors_df = pd.DataFrame()

for r in range(0,len(results_sectors)):
    r_ = results_sectors[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        i_ = np.array(r_)[0]
        sectors_df = pd.concat([sectors_df,pd.DataFrame([s_],index=[i_],columns=['Sector'])],axis=0)

missing_sectors = list(set(SP1500).difference([r[0] for r in results_sectors]))

for r in range(0,len(missing_sectors)):
    r_ = missing_sectors[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    sectors_df = pd.concat([sectors_df,pd.DataFrame(['error'],index=[r_],columns=['Sector'])],axis=0)

In [19]:
#sectors_df

In [20]:
#mktcap_df

In [21]:
#np.sum(sectors_df.index.duplicated())

In [22]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [23]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getMarketCap, npa)
results_mktcp = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_mktcp.append("error")
    else:
        results_mktcp.append(f.result()) 
client.restart()
client.close()


In [24]:
#results_mktcp[0]

In [25]:
#np.array([r[0] for r in results_mktcp])
#names_mktcap = [n[0] for n in results_mktcp]

mktcap_df = pd.DataFrame()

for r in range(0,len(results_mktcp)):
    r_ = results_mktcp[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        #print(s_[0])
        i_ = np.array(r_)[0]
        mktcap_df = pd.concat([mktcap_df,pd.DataFrame([s_[0]],index=[i_],columns=['Market Cap'])],axis=0)

missing_mktcaps = list(set(SP1500).difference([n[0] for n in results_mktcp]))

for r in range(0,len(missing_mktcaps)):
    r_ = missing_mktcaps[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    mktcap_df = pd.concat([mktcap_df,pd.DataFrame([np.nan],index=[r_],columns=['Market Cap'])],axis=0)


C:\Users\User\AppData\Local\Temp/ipykernel_21460/2928409512.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  s_ = np.array(r_)[1]
C:\Users\User\AppData\Local\Temp/ipykernel_21460/2928409512.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  i_ = np.array(r_)[0]


In [26]:
mktcap_df

,Market Cap
NWE,3.355174e+09
WOR,2.349591e+09
FICO,1.111435e+10
TCMD,1.948601e+08
LTHM,4.402919e+09
...,...
V,NaN
SBUX,NaN
WBD,NaN
APA,NaN


In [27]:
mktcap_df.sort_values(by='Market Cap',ascending=False)

,Market Cap
AAPL,2.439796e+12
MSFT,2.045324e+12
GOOG,1.515107e+12
GOOGL,1.512033e+12
AMZN,1.234236e+12
...,...
V,NaN
SBUX,NaN
WBD,NaN
APA,NaN


In [28]:
sectors_df.loc[~sectors_df.index.duplicated(), :].reindex(mktcap_df.loc[~mktcap_df.index.duplicated(), :].index)

,Sector
NWE,Utilities
WOR,Industrials
FICO,Technology
TCMD,Healthcare
LTHM,Basic Materials
...,...
V,Financial Services
SBUX,Consumer Cyclical
WBD,Communication Services
APA,Energy


In [29]:
pd.concat([sectors_df,mktcap_df],axis=1)

,Sector,Market Cap
PNC,Financial Services,7.288021e+10
MMM,Industrials,8.437496e+10
CBRE,Real Estate,2.663606e+10
WOR,Industrials,2.349591e+09
LVS,Consumer Cyclical,2.686189e+10
...,...,...
CADE,error,5.092842e+09
RLGY,error,1.428203e+09
LHCG,error,5.258370e+09
MSEX,error,1.479196e+09


In [30]:
#total_mkt_cap = mktcap_df.replace([np.nan], 0).sum()
#mktcap_df['mkt_cap_pct'] = mktcap_df.replace([np.nan], 0)/total_mkt_cap

In [31]:
#[rp[2] for rp in results_prices][0].info['sector']

In [32]:
"""
arr=np.array(names_prices)
pd.Series(arr).value_counts()
set_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]
if(len(set_)==0):
    duplicates = []
    pass
else:
    duplicates = set_.index[0]
    """

'\narr=np.array(names_prices)\npd.Series(arr).value_counts()\nset_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]\nif(len(set_)==0):\n    duplicates = []\n    pass\nelse:\n    duplicates = set_.index[0]\n    '

In [33]:
#len(results)

In [34]:
names_prices = [rp[0] for rp in results_prices]

In [37]:
prices_df = pd.DataFrame()

#def process_df(data_):
    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    data_ = results_prices[n][1]
    data_['Symbol'] = names_prices[n]
    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['High']).rolling(252).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).min()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).min()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_risk_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_min'] = (pd.DataFrame(data_)['High']).rolling(40).min()
    
    data_['risk'] = data_['trailing_risk_max']-data_['trailing_risk_min']
    
    data_['momentum_1yr'] = (data_['trailing_1yr_max']-data_['trailing_1yr_min']) + (data_['trailing_60d_max']-data_['trailing_60d_min']) + (data_['trailing_30d_max']-data_['trailing_30d_min']) + (data_['trailing_risk_max']-data_['trailing_risk_min'])
    
    data_['risk_per_share'] = data_['momentum_1yr']/data_['risk']
    
    prices_df = pd.concat([prices_df,data_],axis=0)

In [ ]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [ ]:
"""
screenerMetrics = []

for n in npa:
    
    #data_df = pd.DataFrame(n[0])
    data_df = n
    n_ = data_df['Symbol'].values[-1]
    #print(n_)
    value = calculateMinMax(n)
    #print(value)
    screenerMetrics.append(value)
"""

In [52]:
npa = []
for s in names_prices:
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMinMax, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [ ]:
#names_prices[np.where(np.array(names_prices)=='MMM')[0][0]]

In [ ]:
#np.min(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['Open'])

In [49]:
#calculateMinMax(npa[np.where(np.array(names_prices)=='GBX')[0][0]])

In [59]:
#pd.DataFrame(screener['risk_trend']).sort_values(by='risk_trend',ascending=False)

,risk_trend
1054,53.026171
1392,31.060938
108,21.902852
714,17.411716
1010,15.649131
...,...
818,NaN
853,NaN
1107,NaN
1143,NaN


In [60]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')


prices_df.to_excel(writer, sheet_name='SP1500Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

sectors_df.to_excel(writer, sheet_name='stock_sectors')

mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
screener = pd.DataFrame(screenerMetrics,columns=['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct','risk_trend'])
screener['2yr_pct'] = (screener['latest_p']-screener['min_2y'])/(screener['max_2y']-screener['min_2y'])
screener['1yr_pct'] = (screener['latest_p']-screener['min_1y'])/(screener['max_1y']-screener['min_1y'])
screener.sort_values(kind="quicksort", ascending=True,by='risk_trend',inplace=True)
screener.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Sectors', attr_text='stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('market_caps', attr_text='market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)

#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"